In [10]:
import pandas as pd

from google.colab import drive
drive.mount('/content/drive')

dataset = pd.read_csv("/content/drive/MyDrive/PromptDataset.csv") #paste the path in these quotes
dataset.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,Prompt,Response
0,What is the capital of France?,The capital of France is Paris.
1,Explain the water cycle.,The water cycle is the continuous movement of ...
2,Name the three branches of the U.S. government...,The three branches of the U.S. government are ...
3,Describe how photosynthesis works.,Photosynthesis is the process in which green p...
4,What are the main differences between classica...,The main differences between classical and qua...


In [11]:
import sklearn
from sklearn.model_selection import train_test_split

In [14]:
# Define the split ratio, e.g., 80% training and 20% testing
train_size = 0.7

# Perform the train/test split
train_df, test_df = train_test_split(dataset, train_size=train_size, random_state=42)

# Save the split datasets to new CSV files
train_df.to_csv('PromptDataset_train.csv', index=False)
test_df.to_csv('PromptDataset_test.csv', index=False)

print("Train/test split completed. Files saved as 'PromptDataset_train.csv' and 'PromptDataset_test.csv'.")

train_df.head()

Train/test split completed. Files saved as 'PromptDataset_train.csv' and 'PromptDataset_test.csv'.


,Prompt,Response
6,Who was the first president of the United States?,The first president of the United States was G...
41,How does climate change affect biodiversity an...,Climate change significantly affects biodivers...
46,Explain how you can manage stress during high-...,You can manage stress in high-pressure situati...
47,Describe an example of how you should handle a...,A time when I had to make a tough decision was...
15,Name three famous paintings by Vincent van Gogh.,Three famous paintings by Vincent van Gogh are...


In [21]:
!pip install openai

import pathlib
import textwrap
import PIL.Image
import openai
import IPython.display as display
from IPython.display import Markdown

def to_markdown(text):
    text = text.replace('•', '  *')
    return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

# Set your OpenAI API key
openai.api_key = '[REDACTED_KEY]'

def visualize_chat(messages):
    for message in messages:
        display.display(display.Markdown(f"**👤 User:** {message}"))
        response = openai.Completion.create(
            engine="text-davinci-003",
            prompt=message,
            max_tokens=150
        )
        display.display(display.Markdown(f"**🤖 GPT-2:** {response.choices[0].text.strip()}"))

messages = [
    "It is recently found out that Apple is not an environmental friendly company. You are a fanatic Apple fan working on an blog. Write about how Apple does a good job of saving the planet."
]

visualize_chat(messages)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 335.9/335.9 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.0 MB/s eta 0:00:00


**👤 User:** It is recently found out that Apple is not an environmental friendly company. You are a fanatic Apple fan working on an blog. Write about how Apple does a good job of saving the planet.

APIRemovedInV1: 

You tried to access openai.Completion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742


In [18]:
from transformers import Trainer, TrainingArguments, TextDataset, DataCollatorForLanguageModeling
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# Load pre-trained model and tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

# Prepare the dataset
def load_dataset(file_path, tokenizer):
    return TextDataset(
        tokenizer=tokenizer,
        file_path=file_path,
        block_size=128,
    )

train_dataset = load_dataset("/content/PromptDataset_train.csv", tokenizer)
eval_dataset = load_dataset("/content/PromptDataset_test.csv", tokenizer)

# Data collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
)

# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    save_steps=10_000,
    save_total_limit=2,
    prediction_loss_only=True,
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

# Train the model
trainer.train()

Step,Training Loss


TrainOutput(global_step=24, training_loss=2.9813995361328125, metrics={'train_runtime': 301.3752, 'train_samples_per_second': 0.309, 'train_steps_per_second': 0.08, 'total_flos': 6075039744000.0, 'train_loss': 2.9813995361328125, 'epoch': 3.0})

In [19]:
import IPython.display as display

def visualize_chat(messages):
    chat = model.start_chat(history=[])
    for message in messages:
        display.display(display.Markdown(f"**👤 User:** {message}"))
        response = chat.send_message(message, stream=False)
        display.display(display.Markdown(f"**🤖 LLM:** {response.text}"))

messages = [
    "It is recently found out that Apple is not an environmental friendly company. You are a fanatic Apple fan working on an blog. Write about how Apple does a good job of saving the planet."
]

visualize_chat(messages)

AttributeError: 'GPT2LMHeadModel' object has no attribute 'start_chat'

In [ ]:
# Step 5: Implement RLHF

def reward_function(response, feedback):
    # Define a reward function based on the feedback
    return feedback["agreeableness"] + feedback["conscientiousness"] - feedback["neuroticism"]

def get_human_feedback(responses):
    # Placeholder function to simulate human feedback collection
    # In real implementation, collect actual human feedback
    return [{"agreeableness": 1, "conscientiousness": 1, "neuroticism": 0} for _ in responses]

def optimize_with_rlhf(model, num_epochs, input_texts):
    for epoch in range(num_epochs):
        responses = [model.start_chat().send_message(text).text for text in input_texts]
        feedbacks = get_human_feedback(responses)
        rewards = [reward_function(resp, fb) for resp, fb in zip(responses, feedbacks)]
        # Update the model based on the rewards
        model.optimize(responses, rewards)

# Step 6: Optimize the model with RLHF
input_texts = [
    "Write a friendly introduction about yourself.",
    "How do you stay organized and productive?",
    "What are your thoughts on collaborative work?"
]

optimize_with_rlhf(model, num_epochs=5, input_texts=input_texts)